In [ ]:
print("https://keras.io/ja/layers/recurrent/#gru")
    

https://keras.io/ja/layers/recurrent/#gru


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import GRU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(123)

def sin(x, T=100):
    return np.sin(2.0 * np.pi * x / T)

def toy_problem(T=100, ampl=0.05):
    x = np.arange(0, 2 * T + 1)
    noise = ampl * np.random.uniform(low=-1.0, high=1.0, size=len(x))
    return sin(x) + noise

'''
データの生成
'''
T = 100
f = toy_problem(T)

length_of_sequences = 2 * T
maxlen = 25  # ひとつの時系列データの長さ
data = []
target = []

for i in range(0, length_of_sequences - maxlen + 1):
    data.append(f[i: i + maxlen])
    target.append(f[i + maxlen])

X = np.array(data).reshape(len(data), maxlen, 1)
Y = np.array(target).reshape(len(data), 1)

# データ設定
N_train = int(len(data) * 0.9)
N_validation = len(data) - N_train
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=N_validation)

'''
モデル設定
'''
n_in = len(X[0][0])  # 1
n_hidden = 80
n_out = len(Y[0])  # 1

def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model = Sequential()
model.add(GRU(n_hidden,
              kernel_initializer=weight_variable,
              input_shape=(maxlen, n_in)))
model.add(Dense(n_out, kernel_initializer=weight_variable))
model.add(Activation('linear'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='mean_squared_error',
              optimizer=optimizer)

'''
モデル学習
'''
epochs = 500
batch_size = 10
model.fit(X_train, Y_train,  batch_size=batch_size,
          epochs=epochs,  validation_data=(X_validation, Y_validation),
          callbacks=[early_stopping])

'''
出力を用いて予測
'''
truncate = maxlen
Z = X[:1]  # 元データの最初の一部だけ切り出し
original = [f[i] for i in range(maxlen)]
predicted = [None for i in range(maxlen)]

for i in range(length_of_sequences - maxlen + 1):
    z_ = Z[-1:]
    y_ = model.predict(z_)
    sequence_ = np.concatenate(
        (z_.reshape(maxlen, n_in)[1:], y_),
        axis=0).reshape(1, maxlen, n_in)
    Z = np.append(Z, sequence_, axis=0)
    predicted.append(y_.reshape(-1))

'''
グラフで可視化
'''
plt.rc('font', family='serif')
plt.figure()
plt.plot(toy_problem(T, ampl=0), linestyle='dotted', color='#aaaaaa')
plt.plot(original, linestyle='dashed', color='black')
plt.plot(predicted, color='black')
plt.show()

Using TensorFlow backend.


Train on 158 samples, validate on 18 samples
Epoch 1/500
158/158 [==============================] - 0s - loss: 0.4920 - val_loss: 0.3407
Epoch 2/500
158/158 [==============================] - 0s - loss: 0.3037 - val_loss: 0.1516
Epoch 3/500
158/158 [==============================] - 0s - loss: 0.1346 - val_loss: 0.0831
Epoch 4/500
158/158 [==============================] - 0s - loss: 0.0728 - val_loss: 0.0407
Epoch 5/500
158/158 [==============================] - 0s - loss: 0.0404 - val_loss: 0.0221ss: 0.041
Epoch 6/500
158/158 [==============================] - 0s - loss: 0.0216 - val_loss: 0.0185
Epoch 7/500
158/158 [==============================] - 0s - loss: 0.0095 - val_loss: 0.0070
Epoch 8/500
158/158 [==============================] - 0s - loss: 0.0039 - val_loss: 0.0034
Epoch 9/500
158/158 [==============================] - 0s - loss: 0.0020 - val_loss: 0.0015
Epoch 10/500
158/158 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0013
Epoch 11/500
158/158 [===

158/158 [==============================] - 0s - loss: 0.0012 - val_loss: 0.0010
Epoch 89/500
158/158 [==============================] - 0s - loss: 0.0012 - val_loss: 0.0013
Epoch 90/500
158/158 [==============================] - 0s - loss: 0.0012 - val_loss: 0.0010
Epoch 91/500
 90/158 [================>.............] - ETA: 0s - loss: 0.0011